
# YOLOv8 Object Detection on a Custom Dataset





In [ ]:
!nvidia-smi

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

## Install YOLOv8

⚠️ YOLOv8 is still under heavy development. Breaking changes are being introduced almost weekly. We strive to make our YOLOv8 notebooks work with the latest version of the library. Last tests took place on **27.01.2023** with version **YOLOv8.0.20**.

If you notice that our notebook behaves incorrectly - especially if you experience errors that prevent you from going through the tutorial - don't hesitate! Let us know and open an [issue](https://github.com/roboflow/notebooks/issues) on the Roboflow Notebooks repository.

YOLOv8 can be installed in two ways - from the source and via pip. This is because it is the first iteration of YOLO to have an official package.

In [ ]:
# Pip install method (recommended)

!pip install ultralytics==8.0.20

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
# Git clone method (for development)

# %cd {HOME}
# !git clone github.com/ultralytics/ultralytics
# %cd {HOME}/ultralytics
# !pip install -qe ultralytics

# from IPython import display
# display.clear_output()

# import ultralytics
# ultralytics.checks()

In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

## CLI Basics 

If you want to train, validate or run inference on models and don't need to make any modifications to the code, using YOLO command line interface is the easiest way to get started. Read more about CLI in [Ultralytics YOLO Docs](https://v8docs.ultralytics.com/cli/).

```
yolo task=detect    mode=train    model=yolov8n.yaml      args...
          classify       predict        yolov8n-cls.yaml  args...
          segment        val            yolov8n-seg.yaml  args...
                         export         yolov8n.pt        format=onnx  args...
```

## Inference with Pre-trained COCO Model

### 💻 CLI

`yolo mode=predict` runs YOLOv8 inference on a variety of sources, downloading models automatically from the latest YOLOv8 release, and saving results to `runs/predict`.

In [ ]:
%cd {HOME}
!yolo task=detect mode=predict model=yolov8n.pt conf=0.25 source='https://media.roboflow.com/notebooks/examples/dog.jpeg' save=True

In [ ]:
%cd {HOME}
Image(filename='runs/detect/predict/dog.jpeg', height=600)

### 🐍 Python SDK

The simplest way of simply using YOLOv8 directly in a Python environment.

In [ ]:
model = YOLO(f'{HOME}/yolov8n.pt')
results = model.predict(source='https://media.roboflow.com/notebooks/examples/dog.jpeg', conf=0.25)

In [ ]:
results[0].boxes.xyxy

In [ ]:
results[0].boxes.conf

In [ ]:
results[0].boxes.cls

## Preparing a custom dataset

Building a custom dataset can be a painful process. It might take dozens or even hundreds of hours to collect images, label them, and export them in the proper format. Fortunately, Roboflow makes this process as straightforward and fast as possible. Let me show you how!

### Step 1: Creating project

Before you start, you need to create a Roboflow [account](https://app.roboflow.com/login). Once you do that, you can create a new project in the Roboflow [dashboard](https://app.roboflow.com/). Keep in mind to choose the right project type. In our case, Object Detection.

<div align="center">
  <img
    width="640"
    src="https://ik.imagekit.io/roboflow/preparing-custom-dataset-example/creating-project.gif?ik-sdk-version=javascript-1.4.3&updatedAt=1672929799852"
  >
</div>

### Step 2: Uploading images

Next, add the data to your newly created project. You can do it via API or through our [web interface](https://docs.roboflow.com/adding-data/object-detection).

If you drag and drop a directory with a dataset in a supported format, the Roboflow dashboard will automatically read the images and annotations together. 

<div align="center">
  <img
    width="640"
    src="https://ik.imagekit.io/roboflow/preparing-custom-dataset-example/uploading-images.gif?ik-sdk-version=javascript-1.4.3&updatedAt=1672929808290"
  >
</div>

### Step 3: Labeling with Roboflow Annotate


<div align="center">
  <img
    width="640"
    src="https://user-images.githubusercontent.com/26109316/210901980-04861efd-dfc0-4a01-9373-13a36b5e1df4.gif"
  >
</div>

### Step 4: Generate new dataset version

Now that we have our images and annotations added, we can Generate a Dataset Version. When Generating a Version, you may elect to add preprocessing and augmentations. This step is completely optional, however, it can allow you to significantly improve the robustness of your model.

<div align="center">
  <img
    width="640"
    src="https://media.roboflow.com/preparing-custom-dataset-example/generate-new-version.gif?ik-sdk-version=javascript-1.4.3&updatedAt=1673003597834"
  >
</div>

### Step 5: Exporting dataset

Once the dataset version is generated, we have a hosted dataset we can load directly into our notebook for easy training. Click `Export` and select the `YOLO v5 PyTorch` dataset format.

<div align="center">
  <img
    width="640"
    src="https://ik.imagekit.io/roboflow/preparing-custom-dataset-example/export.gif?ik-sdk-version=javascript-1.4.3&updatedAt=1672943313709"
  >
</div>




In [ ]:
# !mkdir {HOME}/datasets
# %cd {HOME}/datasets

# !pip install roboflow --quiet

# from roboflow import Roboflow
# rf = Roboflow(api_key="YOUR_API_KEY")
# project = rf.workspace("roboflow-jvuqo").project("football-players-detection-3zvbc")
# dataset = project.version(1).download("yolov8")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mkdir dataset

In [ ]:
!unzip /content/drive/MyDrive/Freelancing_task/Upwork/Pole_detection/pole_detection.v1i.yolov8.zip -d /content/dataset

## Custom Training

In [ ]:
%cd {HOME}

!yolo task=detect mode=train model=yolov8s.pt 
data= /content/dataset/data.yaml epochs=25 imgsz=800 plots=True

In [ ]:
!ls {HOME}/runs/detect/train/

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train6/confusion_matrix.png', width=1200)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train5/results.png', width=1200)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/val/val_batch0_pred.jpg', width=800)

## Validate Custom Model

In [ ]:
%cd {HOME}

!yolo task=detect mode=val model=/content/runs/detect/train/weights/best.pt data=/content/dataset/data.yaml

## Inference with Custom Model

In [ ]:
# %cd {HOME}
!yolo task=detect mode=predict model="C:\Users\andre\OneDrive\Escritorio\Pole Detection\runs\detect\train4\weights\best.pt" conf=0.25 source="C:\Users\andre\OneDrive\Escritorio\Pole Detection\datasets\test\images" save=True


In [ ]:
from ultralytics import YOLO

model = YOLO("C:\\Users\\andre\\OneDrive\\Escritorio\\Pole Detection\\runs\\detect\\train4\\weights\\best.pt")
results = model.predict(source="C:\\Users\\andre\\OneDrive\\Escritorio\\Pole Detection\\datasets\\test\\images", conf=0.25, save=True)



In [ ]:
import glob
import matplotlib.pyplot as plt
from matplotlib.image import imread

image_paths = glob.glob(f'{HOME}\\runs\\detect\\predict5\*.jpg')[:3]

fig, axes = plt.subplots(1, len(image_paths), figsize=(150, 150))
fig.tight_layout()

for idx, image_path in enumerate(image_paths):
    img = imread(image_path)
    axes[idx].imshow(img)
    axes[idx].axis('off')

plt.show()


# # Save the best model
!copy "C:\\Users\\andre\\OneDrive\\Escritorio\\Pole Detection\\runs\\detect\\train10\\weights\\best.pt" "C:\\Users\\andre\\OneDrive\\Escritorio\\Pole Detection\\"



In [ ]:
!cd

In [ ]:
!jupyter nbconvert --to script --output C:\\Users\\andre\\Downloads\\run_1.py train_yolov8_object_detection_on_custom_dataset.ipynb


In [ ]:
results_df.columns = results_df.columns.str.strip()

precision = results_df['metrics/precision(B)'].mean()
recall = results_df['metrics/recall(B)'].mean()
f1 = 2 * precision * recall / (precision + recall)
mAP50 = results_df['metrics/mAP50(B)'].mean()

print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1 Score: {f1:.3f}")
print(f"mAP50: {mAP50:.3f}")


In [ ]:
import pandas as pd

# Load the results CSV file into a DataFrame
results_df = pd.read_csv('C:\\Users\\andre\\Downloads\\Pole Detection\\runs\\detect\\train6\\results.csv')

# Calculate the average precision and recall
avg_precision = results_df.iloc[1:, 4].astype(float).mean().round(3)
avg_recall = results_df.iloc[1:, 5].astype(float).mean().round(3)
mAP50 = results_df.iloc[1:, 6].astype(float).mean().round(3)
mAP50_95 = results_df.iloc[1:, 7].astype(float).mean().round(3)

# Print the results
print(f"Average Precision: {avg_precision}")
print(f"Average Recall: {avg_recall}")
print(f"mAP50: {mAP50}")
print(f"mAP50_95: {mAP50_95}")